# Portfolio 1

1号组合是一个被动组合，主要关注沪深300、中证500、科创50、北证50、MSCI中国A50指数。

因为还没有找到合适的券商交易API，为便于操作和计算，选取了对应的场外ETF联接基金，这样可以通过在支付宝中设置定投实现买入操作，分别是：

- 沪深300：易方达沪深300ETF联接C(007339)
- 中证500：易方达中证500ETF联接C(007029)
- 科创50：易方达上证科创50联接C(011609)
- 北证50：易方达北证50成份指数C(017516)
- MSCI中国A50：易方达MSCI中国A50互联互通ETF联接C(014533)

## 交易策略

为方便计算，假设本金10万元，5个指数均等分配，每个指数2万元。再将每个指数的2万元分成100份进行定投。

### 买入

1. 沪深300和MSCI中国A50每周二、四定投2次，采用涨跌幅策略。
2. 中证500、科创50和北证50每周二、三、四定投3次，采用涨跌幅策略。

### 卖出

每个交易日14:30检查指数成交价或基金估算净值：

1. 当投入份额小于25%时，持有收益大于6%，则全部卖出。
2. 当投入份额小于50%时，持有收益大于8%，则全部卖出。
3. 当投入份额小于75%时，持有收益大于10%，则全部卖出。
4. 当投入份额大于75%时，持有收益大于12%，则全部卖出。

## 数据源

1. [中证指数](https://www.csindex.com.cn/#/indices/family/detail?indexCode=000300)
2. [蛋卷基金](https://danjuanfunds.com/djmodule/value-center?channel=1300100141)
3. [雪球基金](https://danjuanfunds.com/funding/011609?channel=XQWEB0001)
4. [东方财富](https://quote.eastmoney.com/sh560050.html)
5. [同花顺](https://fund.10jqka.com.cn/011609/)

### 指数数据

1. 沪深300：https://www.csindex.com.cn/csindex-home/perf/index-perf-oneday?indexCode=000300

In [ ]:
import requests

url = 'https://www.csindex.com.cn/#/indices/family/detail?indexCode=000300'
headers = headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0"
}
# 访问网页
response = requests.get('https://www.csindex.com.cn/#/indices/family/detail?indexCode=000300')

# 获取 cookies
cookies = response.cookies

csi_url = "https://www.csindex.com.cn/csindex-home/perf/index-perf-oneday?indexCode=000300"

# 访问网页
response = requests.get(csi_url, headers=headers, cookies=cookies)

print(response.content.decode('utf-8'))

In [ ]:
from playwright.sync_api import sync_playwright

with sync_playwright() as p:
    browser = p.chromium.launch()
    context = browser.new_context()
    page = context.new_page()
    page.goto('https://www.csindex.com.cn/#/indices/family/detail?indexCode=000300')
    
    # 获取 cookies
    cookies = context.cookies()
    print(cookies)

    browser.close()

In [ ]:
%autoawait
import asyncio
from playwright.async_api import async_playwright

async with async_playwright() as p:
    browser = await p.chromium.launch()
    context = await browser.new_context()
    page = await context.new_page()
    await page.goto('https://www.csindex.com.cn/#/indices/family/detail?indexCode=000300')
    # 获取 cookies
    cookies = await context.cookies()
    print(cookies)
    await browser.close()

In [ ]:
import httpx

# 请求URL
url = "https://www.csindex.com.cn/csindex-home/perf/index-perf-oneday?indexCode=000300&type__1773=n4Uxn7e7qmqGw%2BDBqDweeqhSDR77DunE70oD"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0"
}
response = httpx.get(url, headers=headers, cookies=cookies)

# 打印响应内容
print(response.json())

2. 中证500：https://www.csindex.com.cn/csindex-home/perf/index-perf-oneday?indexCode=000905

3. 科创50：https://www.csindex.com.cn/csindex-home/perf/index-perf-oneday?indexCode=000688

In [ ]:
import httpx

# 请求URL
url = "https://www.csindex.com.cn/csindex-home/perf/index-perf-oneday?indexCode=000300&type__1773=n4Uxn7e7qmqGw%2BDBqDweeqhSDR77DunE70oD"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0"
}
response = httpx.get(url)

# 打印响应内容
print(response.json())

4. 北证50：https://www.csindex.com.cn/csindex-home/perf/index-perf-oneday?indexCode=899050

### 基金净值

1. https://fund.eastmoney.com/007029.html?spm=search
2. https://xueqiu.com/S/SH560050
3. https://danjuanfunds.com/djapi/fund/nav/history/011609?page=1&size=20
4. https://fundgz.1234567.com.cn/js/011609.js

In [ ]:
import httpx

# 请求URL
url = "https://fund.10jqka.com.cn/data/client/myfund/011609"
response = httpx.get(url)

# 打印响应内容
print(response.json())

In [ ]:
import json
from datetime import datetime

# 获取当前日期
current_date = datetime.now().strftime('%Y-%m-%d')

# 读取 JSON 文件
with open('../data/trade_day.json', 'r') as f:
    data = json.load(f)

# 获取当前年份
current_year = datetime.now().strftime('%Y')

# 检查当前日期是否在文件中
if current_year in data and any(item['t_dt'] == current_date for item in data[current_year]):
    print('当前日期在文件中。')
else:
    print('当前日期不在文件中。')

In [ ]:
import requests
import time

# 获取当前的毫秒时间戳
timestamp_ms = int(time.time() * 1000)
index_code = '000688'
url = f'https://push2.eastmoney.com/api/qt/stock/get?invt=2&fltt=1&cb=jQuery35103551041611965715_{timestamp_ms}&secid=1.{index_code}'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0"
}
print(url)
response = requests.get(url, headers=headers)
print(response.content.decode('utf-8'))

In [ ]:
import json
import re

response_text = response.content.decode('utf-8')

# 使用正则表达式删除函数调用
json_text = re.sub(r'^\w+\((.*)\);$', r'\1', response_text)

# 解析 JSON 数据
data = json.loads(json_text)

print(data['data']['f60'])

In [ ]:
url = f"https://fundgz.1234567.com.cn/js/017516.js"
headers = {
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0"
}
print(url)
response = requests.get(url, headers=headers)
response_text = response.content.decode("utf-8")
print(response_text)
# 使用正则表达式删除函数调用
json_text = re.sub(r'^\w+\((.*)\);$', r'\1', response_text)

# 解析 JSON 数据
data = json.loads(json_text)
print(data)

f43：当前值
f44：最高
f45：最低
f46：今开
f47：成交量（手）
f48：成交额
f60：昨收
f85：流通股

In [36]:
import json

def load_json_data(fund_code: str):
    json_file = f"../data/{fund_code}.json"
    with open(json_file, "r") as f:
        return json.load(f)


def get_market_data(fund_code: str, date: str):
    data = load_json_data(fund_code)
    return next((item for item in data["market_data"] if item["date"] == date), None)


def get_last_market_data(fund_code: str):
    data = load_json_data(fund_code)
    return data["market_data"][-1]

get_market_data("007339", "2024-02-28")

涨跌幅模式定投分段：

In [ ]:
{
    "25% <= 涨幅": "50%",
    "20% <= 涨幅 < 25%": "52.5%",
    "15% <= 涨幅 < 20%": "55%",
    "10% <= 涨幅 < 15%": "60%",
    "7.5% <= 涨幅 < 10%": "70%",
    "5% <= 涨幅 < 7.5%": "80%",
    "2.5% <= 涨幅 < 5%": "90%",
    "-2.5% <= 涨幅 < -2.5%": "100%",
    "-5% <= 涨幅 < -5%": "120%",
    "-7.5% <= 涨幅 < -7.5%": "140%",
    "-10% <= 涨幅 < -7.5%": "160%",
    "-15% <= 涨幅 < -10%": "180%",
    "-20% <= 涨幅 < -15%": "190%",
    "-25% <= 涨幅 < -20%": "195%",
    "涨幅 < -25%": "200%"
}


In [29]:
from datetime import datetime
import json
import copy

def get_market_data(fund_code: str, date: str):
    json_file = f"../data/{fund_code}.json"
    with open(json_file, 'r') as f:
        data = json.load(f)

    for item in data["market_data"]:
        if item["date"] == date:
            return item
        
    return None


def update_change_data(date: str):
    file_path = "../data/portfolio-1.json"
    with open(file_path, 'r+', encoding="utf-8") as f:
        data = json.load(f)
        last_date_change = data["change_records"][-1]
        last_date = last_date_change["date"]
        
        if date <= last_date:
            return data
        
        days = (datetime.strptime(date, "%Y-%m-%d") - datetime.strptime(last_date, "%Y-%m-%d")).days
        trade_data = next((item for item in data["trade_records"] if item["date"] == date), None)
        
        new_record = copy.deepcopy(last_date_change)
        new_record["date"] = date

        if trade_data:
            new_record["balance"] = round(last_date_change["balance"] * (1 + 0.02 * days / 365) - trade_data.get("trade_money", 0) - trade_data.get("trade_fee", 0), 2)
            new_record["fund_value_total"] = 0
            new_record["fund_cost_total"] = 0
            
            for item in new_record["fund_detail"]:
                trade_item = next((trade for trade in trade_data["trade_detail"] if item["fund_code"] == trade["fund_code"]), None)
                if trade_item and trade_item["trade_type"] == "buy":
                    item["share"] = round(item["share"] + trade_item.get("trade_share", 0), 4)
                    item["unit_value"] = trade_item.get("trade_price", 0)
                    item["value"] = round(item["share"] * item["unit_value"], 2)
                    item["cost"] += trade_item.get("trade_money", 0)
                    new_record["fund_value_total"] = round(new_record["fund_value_total"] + item["value"])
                    new_record["fund_cost_total"] = round(new_record["fund_cost_total"] + item["cost"])

        else:
            new_record["balance"] = round(last_date_change["balance"] * (1 + 0.02 * days / 365), 2)
            new_record["fund_value_total"] = 0

            for item in new_record["fund_detail"]:
                    item["unit_value"] = float(get_market_data(item["fund_code"], date)["unit_value"])
                    item["value"] = round(item["share"] * item["unit_value"], 2)
                    new_record["fund_value_total"] = round(new_record["fund_value_total"] + item["value"])
            

        # 添加新记录到 change_records 列表
        data["change_records"].append(new_record)

        # 将修改后的数据写回文件
        f.seek(0)
        json.dump(data, f, indent=4)
        f.truncate()

    return data
        
    
update_change_data("2024-02-26")

{'name': 'passive-1',
 'index_code': ['000300', '000905', '000688', '899050'],
 'fund_code': ['007339', '007029', '011609', '017516', '014533'],
 'start_date': '2024-02-19',
 'start_money': 100000,
 'change_records': [{'date': '2024-02-19',
   'balance': 100000,
   'fund_value_total': 0,
   'fund_cost_total': 0,
   'fund_detail': [{'fund_code': '007339',
     'value': 0,
     'share': 0,
     'unit_value': 0,
     'cost': 0},
    {'fund_code': '007029',
     'value': 0,
     'share': 0,
     'unit_value': 0,
     'cost': 0},
    {'fund_code': '011609',
     'value': 0,
     'share': 0,
     'unit_value': 0,
     'cost': 0},
    {'fund_code': '017516',
     'value': 0,
     'share': 0,
     'unit_value': 0,
     'cost': 0},
    {'fund_code': '014533',
     'value': 0,
     'share': 0,
     'unit_value': 0,
     'cost': 0}]},
  {'date': '2024-02-20',
   'balance': 99005.48,
   'fund_value_total': 1000.0,
   'fund_cost_total': 1000,
   'fund_detail': [{'fund_code': '007339',
     'value':

In [ ]:
"2024-02-19" > "2024-02-20"

In [38]:
def is_tuesday_wednesday_thursday(date_str: str, date_format: str = "%Y-%m-%d"):
    # 将字符串转换为 datetime 对象
    date = datetime.strptime(date_str, date_format)
    # 获取日期是星期几
    weekday = date.weekday()
    # 检查是否为周二、周三或周四
    return weekday in [1, 2, 3]

is_tuesday_wednesday_thursday("2024-02-19")

False